In [2]:
import igl
import meshplot as mp
import numpy as np

# Hello world

In [3]:
v, f = igl.read_triangle_mesh("data/bunny.obj")
mp.plot(v, f, v[:, 0])

C:\Anaconda3\envs\AF-course\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0002189…

# Data Representation
Meshes are represented by a numpy array of vertex coordinates (nx3) and a numpy array of face indices (mx3) and can be loaded from 3D file formats

In [4]:
v: np.array # Array of vertex coordinates (nx3)
f: np.array # Array of face indices (mx3)
v, f = igl.read_triangle_mesh("data/bunny.obj")
print(v)
print(f)

[[ 0.579785 -0.556431  0.335583]
 [ 0.566629 -0.530296  0.334038]
 [ 0.533192 -0.543823  0.342491]
 ...
 [-0.521482  0.641195 -0.549294]
 [-0.555727  0.655425 -0.519703]
 [-0.534968  0.676574 -0.551021]]
[[   0    1    2]
 [   0    3    1]
 [   1    4    2]
 ...
 [3296 3298 3300]
 [3299 3300 3298]
 [3299 3295 3300]]


# Visualizing Surfaces and Pointclouds

In [5]:
mp.plot(v, f, c=v[:, 0])
mp.plot(v, c=v[:, 0],shading={"point_size":0.3})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0002189…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0002189…

# Gauss map of the input mesh

In [6]:
# Loading the mesh
# v, f = igl.read_triangle_mesh("data/sphere.obj")
v, f = igl.read_triangle_mesh("data/hemisphere.obj")
# v, f = igl.read_triangle_mesh("data/CrumpledDevelopable.obj")
# v, f = igl.read_triangle_mesh("data/cylinder.obj")
# v, f = igl.read_triangle_mesh("data/bunny.obj")
# v, f = igl.read_triangle_mesh("data/face.off")
mp.plot(v,f,c=v[:,0])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [7]:
# Calculating per face normals
nf = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
# Plotting face normals
mp.plot(v,f,c=nf)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [8]:
# Calculating per vertex normals
nv = igl.per_vertex_normals(v, f)
avg = igl.avg_edge_length(v, f)
# Plotting vertex normals as lines
pv=mp.plot(v,f, c=nv)
pv.add_lines(v, v + nv*avg)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

1

In [9]:
# Calculating the principal curvature, mean curvature and gassian curvature.
v1, v2, k1, k2 = igl.principal_curvature(v, f)
mean_curv = 0.5 * (k1 + k2)
gaussian_curv=k1*k2
avg = igl.avg_edge_length(v, f)

# Plotting gaussian curvature + directions
pgc=mp.plot(v, f, c=gaussian_curv)
pgc.add_lines(v + v1 * avg/3, v - v1 * avg/3, shading={"line_color": "red"})
pgc.add_lines(v + v2 * avg/3, v - v2 * avg/3, shading={"line_color": "green"})



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

2

In [10]:
# Plotting the gauss map
vs, fs = igl.read_triangle_mesh("data/unit_sphere.obj")
vs=vs*0.95
pgm=mp.plot(vs, fs,shading={"flat":False})
pgm.add_points(nf,shading={"point_size":0.2})
ev, fe, ef = igl.edge_topology(v, f)
ef = ef[np.all(ef != -1, axis=1)]
pgm.add_lines(nf[ef[:,0]],nf[ef[:,1]],shading={"line_color": "red"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(4.7683715…

2

# How to calculate the developability on the surface?

In [11]:
vf,ni=igl.vertex_triangle_adjacency(f,v.shape[0])

In [12]:
developability=np.zeros(v.shape[0])
for i in range(0,v.shape[0]):
    dmat=nf[vf[ni[i]:ni[i+1]]]
    developability[i]=min(np.linalg.eigvals(np.dot(dmat.T,dmat)))
pd=mp.plot(v,f, c=developability,shading={"point_size":0.3})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [13]:
max(developability)

0.0796325810514155

In [ ]:
# # cuts=np.zeros(f.shape[0]*3)
# cuts=f*0
# cuts[0:20,0]=1
# vcut,fcut=igl.cut_mesh(v,f,cuts)
# vf,ni=igl.vertex_triangle_adjacency(fcut,vcut.shape[0])
# developability=np.zeros(vcut.shape[0])
# nf = igl.per_face_normals(vcut, fcut, np.array([0.0, 0.0, 0.0]))
# for i in range(0,vcut.shape[0]):
#     dmat=nf[vf[ni[i]:ni[i+1]]]
#     developability[i]=min(np.linalg.eigvals(np.dot(dmat.T,dmat)))
# pd=mp.plot(vcut,fcut, c=developability,shading={"point_size":0.3})

In [19]:
cuts

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0,